## Chapter 9
# Defining new types

## Exercises

**9-0.** Compile, execute and test the programs in this chapter.

Implemented in `Student_info.cpp`, `Student_info.h` and `grading.cpp`.

Compile with `gcc Student_info.cpp grade.cpp grading.cpp ../chapter_4/median.cpp -o grading.out -lstdc++`.

**9-1.** Reimplement the `Student_info` class so that it calculates the final grade when reading the student's record, and stores that grade in the object. Reimplement the `grade` function to use this precomputed value.

Implemented in the above files.  I took the liberty of making the `grade()` method `private` and exposing a `get_total_grade()` `const` method.

**9-2.** If we define the `name` function as a plain, non`const` member function, what other functions in our system must change and why?

 If we define the name function as a non`const` member function, we would also have to remove the `const` declarations from the `compare` `Student_info` parameters, since only `const` member methods can be called on `const` instances.

**9-3.** Our `grade` function was written to throw an exception if a user tried to calculate a grade for a `Student_info` object whose values had not yet been read. Users who care are expected to catch this exception. Write a program that triggers the exception but does not catch it. Write a program that catches the exception.

Implemented in `no_catch.cpp` and `catch.cpp`:

```
$ gcc Student_info.cpp grade.cpp catch.cpp ../chapter_4/median.cpp -o catch.out -lstdc++
$ ./catch.out
Karl 80 90
Caught an error!
student has done no homework
$ gcc Student_info.cpp grade.cpp no_catch.cpp ../chapter_4/median.cpp -o no_catch.out -lstdc++
$ ./no_catch.out
Karl 80 90
libc++abi.dylib: terminating with uncaught exception of type std::domain_error: student has done no homework
[1]    26957 abort      ./no_catch.out
```

**9-4.** Rewrite your program from the previous exercise to use the `valid` function, there by avoiding the exception altogether.

In order to build ontop of exercise **9-1**, I added the `valid` check within the `read` function, before calling `grade`, defaulting to a `total_grade` of `0` if no HW is present:

`    total_grade = valid() ? grade() : 0;`

Here is a run with this version:
```
$ ./grading.out
Karl 80 90
Karl 0
```

**9-5.** Write a class and associated functions to generate grades for students who take the course for pass/fail credit. Assume that only the midterm and final grades matter, and that a student passes with an average exam score greater than 60. The report should list the students in alphabetical order, and indicate `P` or `F` as the grade.

I really don't think an entire new class is called for here.  Even as an exercise it seems ill-defined and strange.  After all, what would the new _object_ even be?  A `PassFailGrader` class? Defining that as an object interface would be counter to the way we're doing things now, which is to leave the responsibility of grading to the `Student_info` object itself.  I took the liberty of implementing this simply as another boolean function `bool passed_for_pass_fail(double midterm, double final)` in `grade.h/cpp`, and added a convenience method `bool Student_info::passed_for_pass_fail()` similar to its `grade` function.

The driver is implemented in `pass_fail_report.cpp`.

```
$ gcc Student_info.cpp grade.cpp pass_fail_report.cpp ../chapter_4/median.cpp -o grading.out -lstdc++
$ ./pass_fail_report.out
Karl 98 50
Jake 50 60
Ken 40 50
Frank 90 90
Jake F
Karl P
Ken  F
rank P
```

**9-6.** Rewrite the grading program for the pass/fail students so that the report shows all the students who passed, followed by all the students who failed.

I used the standard library `partition` function for this task:
```
bool passing(const Student_info& student) {
    return student.passed_for_pass_fail();
}

...

    vector<Student_info>::const_iterator failing_begin = partition(students.begin(), students.end(), passing);

    for (vector<Student_info>::const_iterator it = students.begin(); it != failing_begin; ++it) {
        cout << it->name() << string(maxlen + 1 - it->name().size(), ' ') << 'P' << endl;
    }

    for (vector<Student_info>::const_iterator it = failing_begin; it != students.end(); ++it) {
        cout << it->name() << string(maxlen + 1 - it->name().size(), ' ') << 'F' << endl;
    }
```

Here is a run:
```
$ ./pass_fail_report.out
Karl 98 50
Jake 50 60
Ken 40 50
Sam 90 90
Karl P
Sam  P
Jake F
Ken  F
```

**9-7.** The `read_hw` function in 4.1.3/57 solves a general proglem (reading a sequence of values into a `vector`) even though its name suggests that it should be part of the implementation of `Student_info`. Of course, we could change its name - but suppose, instead, that you wanted to integrate it with the rest of the `Student_info` code, in order to clarify that it was not intended for public access despite its apparent generality? How would you do so?

In order to make the `read_hw` function a member of `Student_info`, the definition and body would change to

```
istream& Student_info::read_hw(istream& in) {
    if (in) {
        homework.clear();

        double x;
        while (in >> x) {
            homework.push_back(x);
        }

        in.clear();
    }
    return in;
}
```

and in `Student_info::read`, we would change the call to `read_hw(is)`.